# Seq2Seq 모델을 활용한 챗봇 만들기

In [1]:
# 우리가 만들 챗봇의 형태도 비슷하게 될것이다.
# '묻고' & '답하는' 형태를 구성.

# Q "나이는 어떻게 되나요?" <- 질문을 인코더 입력
# A "저는 18살입니다" <-  디코더 출력


# '학습' 시 질문에 대한 압축된 정보가 context vector 안에 함축적으로 답겨 있을것이다.


# 기본 import

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras

import random
def set_seed(seed = 42):
  tf.keras.utils.set_random_seed(seed)
  tf.config.experimental.enable_op_determinism()

set_seed(42)

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import re

In [3]:
base_path = r'/content/drive/MyDrive/KoreaIT (코리아it)/250715 💚자연어처리S4/[자연어처리]/dataset(NLP2507)'

In [4]:
corpus = pd.read_csv(os.path.join(base_path, 'chatbot_data', 'chatbot_data.csv'))
corpus.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [5]:
corpus.shape

(11823, 3)

In [6]:
corpus = corpus.head(5000)

# 데이터 전처리

## question 고 answer 의 데이터셋 분리

In [7]:
texts = corpus['Q'].tolist()  # 질문들
pairs = corpus['A'].tolist()  # 답변들

In [8]:
list(zip(texts, pairs))[:5]

[('12시 땡!', '하루가 또 가네요.'),
 ('1지망 학교 떨어졌어', '위로해 드립니다.'),
 ('3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('3박4일 정도 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('PPL 심하네', '눈살이 찌푸려지죠.')]

## 특수문제 제거

In [9]:
# 한글과 숫자를 제외한 특수 문자 제거

In [10]:
import re

In [11]:
def clean_sentence(sentence):
  sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', sentence)
  return sentence

In [12]:
# 동작확인
clean_sentence('12시 땡^^!???')

'12시 땡'

In [13]:
clean_sentence('abcef가나다^^$%@12시 땡^^!??')

'가나다12시 땡'

## Konlpy 형태소분석기

In [14]:
!apt-get update

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,130 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,982 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubu

In [15]:
!apt-get install -y openjdk-11-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core ca-certificates-java fonts-dejavu-core fonts-dejavu-extra
  gsettings-desktop-schemas java-common libatk-bridge2.0-0 libatk-wrapper-java
  libatk-wrapper-java-jni libatk1.0-0 libatk1.0-data libatspi2.0-0
  libpcsclite1 libxcomposite1 libxt-dev libxtst6 libxxf86dga1
  openjdk-11-jdk-headless openjdk-11-jre openjdk-11-jre-headless
  session-migration x11-utils
Suggested packages:
  default-jre pcscd libxt-doc openjdk-11-demo openjdk-11-source visualvm
  libnss-mdns fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  | fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  at-spi2-core ca-certificates-java fonts-dejavu-core fonts-dejavu-extra
  gsettings-desktop-schemas java-common libatk-bridge2.0-0 libatk-wrapper-java
  libatk-wrapper-java-jni libatk1.0-0 libatk1.0-data libatspi2.0

In [16]:
!pip install konlpy JPype1==1.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.9/797.9 kB 44.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 110.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for JPype1
  Running setup.py clean for JPype1
Failed to build JPype1
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (JPype1)


In [17]:
!ls /usr/lib/jvm/java-1.11.0-openjdk-amd64/

bin  conf  docs  include  jmods  legal	lib  man  release


In [18]:
import os
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-1.11.0-openjdk-amd64"
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['JAVA_HOME'], 'bin')

In [20]:
!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 29.9 MB/s eta 0:00:00


In [21]:
from konlpy.tag import Okt

In [22]:
okt = Okt()

In [23]:
print(okt.morphs("정말 힘든 하루입니다"))

['정말', '힘든', '하루', '입니다']


In [24]:
def process_morph(sentence):
  return ' '.join(okt.morphs(sentence))

In [25]:
process_morph('한글은 홀소리와 닿소리 모두 소리틀을 본떠 만든 음소문자[1]로 한글 맞춤법에서는 닿소리 14개와 홀소리 10개, 모두 24개를 표준으로 삼는다. "나랏말이 중국과 달라" 문제를 느낀 조선의 세종대왕이 한국어는 물론 이웃나라 말까지 나타내도록 1443년 창제하여 1446년 반포하였다. ')

'한글 은 홀소리 와 닿소리 모두 소 리틀 을 본떠 만든 음소문자 [ 1 ] 로 한글 맞춤법 에서는 닿소리 14 개 와 홀소리 10 개 , 모두 24 개 를 표준 으로 삼는다 . " 나랏말 이 중국 과 달라 " 문제 를 느낀 조선 의 세종대왕 이 한국어 는 물론 이웃 나라 말 까지 나타내도록 1443년 창제 하여 1446년 반포 하였다 .'

# 데이터셋 구성

**Seq2Seq** 모델이 학습하기 위한 데이터셋을 구성할 때, 다음과 같이 **3가지 데이터셋**을 구성합니다.

- `question`: encoder input 데이터셋 (질의 전체)
- `answer_input`: decoder input 데이터셋 (답변의 시작). **START 토큰**을 문장 처음에 추가 합니다.
- `answer_output`: decoder output 데이터셋 (답변의 끝). **END 토큰**을 문장 마지막에 추가 합니다.

## 문장 입력받아 3가지 데이터셋 구성하는 함수

In [26]:
def clean_and_morph(sentence, is_question=True):
  sentence = clean_sentence(sentence)
  sentence = process_morph(sentence)

  if is_question:
    # QUESTION 인 경우
    return sentence
  else:
    # ANSWER 인 경우
    # (decoder 의 입력, decoder 의 output)
    return ('<START> ' + sentence, sentence + ' <END>')

In [27]:
clean_and_morph("12시 땡!")

'12시 땡'

In [28]:
clean_and_morph("12시 땡!", False)

('<START> 12시 땡', '12시 땡 <END>')

## `text` 와 `pair` 에 대한 데이터셋 구성

In [29]:
# text 와 pair 에 대한 데이터 셋 구성 함수.

# 매개변수
#  texts=  질의
#  pairs=  답변

def preprocess(texts, pairs):
    questions = []  # encoder 의 입력
    answer_in = []  # decoder 의 입력
    answer_out = [] # decoder 의 출력 (타겟)

    # '질의' 에 대한 전처리
    for text in texts:
      question = clean_and_morph(text, is_question=True)
      questions.append(question)

    # '답변' 에 대한 전처리
    for pair in pairs:
      in_, out_ = clean_and_morph(pair, is_question=False)
      answer_in.append(in_)
      answer_out.append(out_)

    return questions, answer_in, answer_out

In [30]:
questions, answer_in, answer_out = preprocess(texts, pairs)

In [31]:
questions[:5]

['12시 땡', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', '심하네']

In [32]:
answer_in[:5]

['<START> 하루 가 또 가네요',
 '<START> 위로 해 드립니다',
 '<START> 여행 은 언제나 좋죠',
 '<START> 여행 은 언제나 좋죠',
 '<START> 눈살 이 찌푸려지죠']

In [33]:
answer_out[:5]

['하루 가 또 가네요 <END>',
 '위로 해 드립니다 <END>',
 '여행 은 언제나 좋죠 <END>',
 '여행 은 언제나 좋죠 <END>',
 '눈살 이 찌푸려지죠 <END>']

## `all_sentences` 변수에 모든 데이터셋 문장을 합침

합치는 이유
1. 단어사전 구축

In [34]:
len(questions)

5000

In [35]:
all_sentences = questions + answer_in + answer_out

In [36]:
len(all_sentences)

15000

In [37]:
# 모든 문장을 공백으로 join 한 뒤에 다시 공백으로 쪼개기
a = (' '.join(questions) + ' '.join(answer_in) + ' '.join(answer_out)).split()

# 총 몇개의 '고유' 단어로 구성되었는지 확인
len(set(a))


6203

# 토큰화 Tokenizer

## 토큰의 option을 정의

- filter는 '(빈 문자열)'로 지정합니다.
- lower는 False로 지정합니다.
- oov_token은 '&lt;OOV&gt;'로 지정합니다.

In [38]:
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')

## 단어 사전 만들기

In [39]:
tokenizer.fit_on_texts(all_sentences)

## word to index

In [40]:
for word, idx in tokenizer.word_index.items():
  print(f'{word}\t\t => \t{idx}')
  if idx > 10: break

<OOV>		 => 	1
<START>		 => 	2
<END>		 => 	3
이		 => 	4
거		 => 	5
을		 => 	6
가		 => 	7
예요		 => 	8
해보세요		 => 	9
도		 => 	10
에		 => 	11


## 단어의 개수

In [41]:
len(tokenizer.word_index)

6202

In [42]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
VOCAB_SIZE

6203

## texts_to_sequences()

In [43]:
# 문장을 정수 인덱스 시퀀스로 변환
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

In [44]:
idx = 0
print(questions[idx])
print(question_sequence[idx])

12시 땡
[3557, 4426]


In [45]:
print(answer_in[idx])
print(answer_in_sequence[idx])

<START> 하루 가 또 가네요
[2, 286, 7, 131, 2153]


## pad_sequences()

In [46]:
MAX_LENGTH = 30
TRUNCATING = 'post'
PADDING = 'post'

# 태스크 마다 pre 로 할지 post 로 할지 달리 해야 한다.
# 	감정분류 <-  pre
# 	번역, 챗봇 <- post

In [47]:
question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating=TRUNCATING, padding=PADDING)
answer_in_padded = pad_sequences(answer_in_sequence, maxlen=MAX_LENGTH, truncating=TRUNCATING, padding=PADDING)
answer_out_padded = pad_sequences(answer_out_sequence, maxlen=MAX_LENGTH, truncating=TRUNCATING, padding=PADDING)

In [48]:
print(question_sequence[0])
print(question_padded[0])

[3557, 4426]
[3557 4426    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [49]:
question_padded.shape

(5000, 30)

# 데이터셋 변환 함수

## 원핫인코딩 to_categorical()

In [50]:
answer_in_one_hot = keras.utils.to_categorical(answer_in_padded, num_classes=VOCAB_SIZE)
answer_out_one_hot = keras.utils.to_categorical(answer_out_padded, num_classes=VOCAB_SIZE)

In [51]:
answer_in_one_hot[0].shape  # (max_len, VOCAB_SIZE)

(30, 6203)

## index_word

In [70]:
def convert_index_to_text(indexes, end_token):
  words = []

  for index in indexes:
    if index == end_token:
      break  # 문장의 끝 토큰이므로 중지

    # 사전에 존재하는 단어의 경우 단어 추가
    if index > 0 and tokenizer.index_word.get(index) is not None:
      words.append(tokenizer.index_word[index])

  return ' '.join(words)

# 모델 생성

In [71]:
# 1. 인코더 모델 만들고
# 2. 디코더 모델 만들고
# 3. 인코더와 디코더 가 연결된 seq2seq 모델을 만들기

In [72]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping

## 학습용 인코더 (Encoder)

![](https://wikidocs.net/images/page/24996/%EC%9D%B8%EC%BD%94%EB%8D%94%EB%94%94%EC%BD%94%EB%8D%94%EB%AA%A8%EB%8D%B8.PNG)

In [73]:
class Encoder(tf.keras.Model):
  def __init__(self, units, vocab_size, embedding_dim):
    super(Encoder, self).__init__()

    self.embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)
    self.dropout = Dropout(0.2)
    self.lstm = LSTM(units, return_state=True, dropout=0.2)

  def call(self, inputs):
    x = self.embedding(inputs)
    x = self.dropout(x)
    x, hidden_state, cell_state = self.lstm(x)

    # encoder 의 목적은 'context 벡터' 를 만들어 내는 거다.
    # x 를 사용하느넥 아니고, [h, c] 묶어서 'context 벡터' 를 리턴.
    return [hidden_state, cell_state]

## 학습용 디코더 (Decoder)

In [74]:
class Decoder(tf.keras.Model):
  def __init__(self, units, vocab_size, embedding_dim):
    super(Decoder, self).__init__()
    self.embedding = Embedding(vocab_size, embedding_dim)
    self.dropout = Dropout(0.2)
    self.lstm = LSTM(units, return_state=True, return_sequences=True, dropout=0.2)
    self.dense = Dense(vocab_size, activation='softmax')

  #initial_state 는 앞선 encodder 만들어낸 context 벡터를 받아오는 거다.
  def call(self, inputs, initial_state):
    x = self.embedding(inputs)
    x = self.dropout(x)
    x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
    x = self.dense(x)
    return x, hidden_state, cell_state

## 하이퍼 파라미터

In [75]:
BATCH_SIZE = 16   # <- 배치크기가 너무 크면, 메모리 부담..  HW 여건에 따라 조정.
EMBEDDING_DIM = 100  # <- 나중에 변경해 보셔도 됨.
TIME_STEPS = MAX_LENGTH

START_TOKEN = tokenizer.word_index['<START>']  # <sos> 토큰 2
END_TOKEN = tokenizer.word_index['<END>']  # <eos> 토큰 3

UNITS = 128  # LSTM units

VOCAB_SIZE = len(tokenizer.word_index)+1   # 위에서 정의하긴 했었다.
DATA_LENGTH = len(questions)  # 몇개의 문장 데이터가 있는지.
SAMPLE_SIZE = 3   # 학습 도중에 몇개만 샘플링 해서 예측 결과값을 지켜볼거다.
NUM_EPOCHS = 20   # fit() 의 epochs= 값이 아니라.   전체 epoch 를 반복할 횟수 지정

## Seq2Seq 모델

![](https://wikidocs.net/images/page/24996/%EC%9D%B8%EC%BD%94%EB%8D%94%EB%94%94%EC%BD%94%EB%8D%94%EB%AA%A8%EB%8D%B8.PNG)

In [76]:
class Seq2Seq(tf.keras.Model):

  def __init__(self, units=UNITS, vocab_size=VOCAB_SIZE, embedding_dim=EMBEDDING_DIM,
               time_steps=TIME_STEPS,
               start_token=START_TOKEN, end_token=END_TOKEN):
    super(Seq2Seq, self).__init__()

    self.start_token = start_token
    self.end_token = end_token
    self.time_steps = time_steps

    self.encoder = Encoder(units, vocab_size, embedding_dim)
    self.decoder = Decoder(units, vocab_size, embedding_dim)

  # Seq2Seq 는 '학습상태'일때와 '예측'할때의 동작이 다르다.
  # inputs= :
  #    학습상태 에선 ('인코더의 입력', '디코더의 입력')
  #    예측상태 에선 '인코더의 입력'
  # training= :
  #    True: '학습상태'  False : '예측상태'
  def call(self, inputs, training=True):
      if training: # '학습상태'
        encoder_inputs, decoder_inputs = inputs
        context_vector = self.encoder(encoder_inputs)
        decoder_outputs, _, _ = self.decoder(inputs=decoder_inputs, initial_state=context_vector)
        return decoder_outputs

      else:  # '예측' 동작
        # 첫번째 timestep 의 입력값 두개 준비

        # 1. 인코더로부터 context 벡터
        context_vector = self.encoder(inputs)

        # 2. 디코더에 입력할 첫번째 토큰 <sos>
        # ↓ target_seq 에는 한단아 한단어 인덱스 들이 담길거다.
        target_seq = tf.constant([[self.start_token]], dtype=tf.float32)

        # ↓ results 는 배열로 만들어
        # 단어 하나하나를 예측하여 배열에 담아줄거다.
        # ※ 주의 call() 안에서는 numpy 배열 사용하면 안된다.
        #    내부적으로 graph 가 형성되어야 하기 때문에 TF 의 TensorArray
        # size=self.time_steps : 한 문장의 최대 길이는 어짜피 MAX_LENGTH 이니
        #    미리 그에 준하는 배열을 준비해둠.
        #    밑에서 results.write() 를 통해 하나하나 추가할거임.
        # 그리고 결국 results 를 리턴한다
        results = tf.TensorArray(dtype=tf.int32, size=self.time_steps)

        for i in tf.range(self.time_steps):
          # 최초 입력되는 target_seq 는 <sos>
          decoder_output, decoder_hidden, decoder_cell =\
              self.decoder(target_seq, initial_state=context_vector)

          # decoder 의 출력을 배열에 담을거다
          # decoder_output 은 softmax 의 결과이기 때문에 argmax() 로 index 변환.
          decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1), dtype=tf.int32)

          # 위 index 를 배열에 쌓기 위해서 벡터 변환
          decoder_output = tf.reshape(decoder_output, shape=(1, 1))

          # results 배열에 쌓기
          results = results.write(i, decoder_output)

          if decoder_output == self.end_token: break  # <eos> 이면 종료

          # 다음 타임스텝에 전달할 입력 토근
          target_seq = decoder_output

          # 다음 타임스텝에 전달할 context 벡터
          context_vector = [decoder_hidden, decoder_cell]


        return tf.reshape(results.stack(), shape=(1, self.time_steps))





# 학습

## callback 준비

In [77]:
out_path = os.path.join(base_path, 'out')
out_path

'/content/drive/MyDrive/KoreaIT (코리아it)/250715 💚자연어처리S4/[자연어처리]/dataset(NLP2507)/out'

In [78]:
checkpoint_path = os.path.join(out_path, 'seq2seq_chatbot_checkpoint.keras')
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_best_only=True,
                             monitor='loss',
                             verbose=1)

## 모델 생성 & compile

In [79]:
seq2seq = Seq2Seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)

In [80]:
seq2seq.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## 예측함수 정의

In [81]:
# 예측함수를 미리 정의해놓고, 학습도중에 일부 샘플을 예측한 결과를 출력해볼거다.

def make_prediction(model, question_inputs):
  results = model(inputs=question_inputs, training=False)  # 모델객체 호출 -> call() 호출

  # results 의 인덱스를 문장으로 변환하기 위해 1차원으로 변환
  results = np.asarray(results).reshape(-1)
  return results # 이 리턴값을 문장으로 변환할 거다.


## 학습 루프

In [82]:
for epoch in range(NUM_EPOCHS):
  print(f'epoch: {epoch * 10 + 1}...')  # fit(epochs=10..) 으로 할거고 이를 NUM_EPOCHS 만큼 반복할거다

  seq2seq.fit([question_padded, answer_in_padded],  # 입력값
              answer_out_one_hot,   # target값
              epochs=10,
              batch_size=BATCH_SIZE,
              callbacks=[checkpoint],
              )

  # 학습 중간에 예측할 샘플 번호들 추출
  samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

  # 질문과 예측값 비교 출력
  for idx in samples:
    question_inputs = question_padded[idx]
    # 예측값 (답변문장의 인덱스)
    results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))

    # 인덱스를 문장으로 변환
    results = convert_index_to_text(results, END_TOKEN)

    print(f'Q: {questions[idx]}')
    print(f'A: {results}')
    print('\n')



epoch: 1...
Epoch 1/10
311/313 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7945 - loss: 3.0593
Epoch 1: loss improved from inf to 1.73799, saving model to /content/drive/MyDrive/KoreaIT (코리아it)/250715 💚자연어처리S4/[자연어처리]/dataset(NLP2507)/out/seq2seq_chatbot_checkpoint.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.7947 - loss: 3.0466
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8367 - loss: 1.1324
Epoch 2: loss improved from 1.73799 to 1.10719, saving model to /content/drive/MyDrive/KoreaIT (코리아it)/250715 💚자연어처리S4/[자연어처리]/dataset(NLP2507)/out/seq2seq_chatbot_checkpoint.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.8367 - loss: 1.1323
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8393 - loss: 1.0627
Epoch 3: loss improved from 1.10719 to 1.04381, saving model to /content/drive/MyDrive/KoreaIT (코리아it)/250715 💚자연어처리S4/[자연어처리]/dataset(NLP2507)/out/seq2seq_chatbot_checkpo

5000

# 예측

## 예측함수 정의
- `make_question` 함수 정의:

자연어 '입력'을 받아서 '데이터 전처리' 파이프라인 수행 후 전처리 진행하는 함수를 정의합니다.

1. 텍스트 정제
1. 형태소 변환
1. 학습에 사용된 Tokenizer 로 정수 인덱싱
1. 패딩

In [87]:
# 자연어 (질문 입력) 에 대한 전처리 함수
def make_question(sentence):
  sentence = clean_and_morph(sentence)
  # 위에서 준비된 Tokenizer 를 사용하여 정수 인덱싱
  question_sequence = tokenizer.texts_to_sequences([sentence])
  # 패딩
  question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH,
                truncating='post', padding='post')

  return question_padded

In [88]:
make_question("3박4일 놀러가고 싶다")

array([[1999, 3349, 3559,  335,  154,   72,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

In [89]:
make_question('커피 마시고 싶다.')

array([[ 176, 1678,   72,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

In [58]:
# ↑ (1, MAX_LENGTH)

## 예측 결과를 자연어로 변환
- `run_chatbot()`함수를 정의합니다. 질문을 `make_question()` 함수 전처리 수행후 모델 예측하여 결과를 다시 자연어로 변환하는 작업을 거치게 됩니다.

In [90]:
def run_chatbot(question, test=False):  # test= : 중간과정 출력여부
  question_inputs = make_question(question)

  test and print('question:', question)
  test and print('make_question:', question_inputs)

  results = make_prediction(seq2seq, question_inputs)

  test and print('make_prediction:', results)

  results = convert_index_to_text(results, END_TOKEN)

  test and print('convert_index_to_text:', results)

  return results

In [91]:
run_chatbot("오늘은 피곤하네", True)

question: 오늘은 피곤하네
make_question: [[62 17  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]]
make_prediction: [  62   17 1688    4   13    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
convert_index_to_text: 오늘 은 예능 이 요


'오늘 은 예능 이 요'

## 챗봇 동작

In [92]:
while True:
  user_input = input("<< 말을 걸어 보세요!\n")
  if user_input == 'q':
    break

  print(f'>> 챗봇 응답 {run_chatbot(user_input)}')

<< 말을 걸어 보세요!
안녕하세요
>> 챗봇 응답 안녕하세요
<< 말을 걸어 보세요!
오늘은 기분이 좋습니다
>> 챗봇 응답 돈 없어도 할 수 있는게 많아요
<< 말을 걸어 보세요!
배가 고파요
>> 챗봇 응답 어떻 게 든 잘 살 거 예요
<< 말을 걸어 보세요!
오늘은 어디로 갈까요?
>> 챗봇 응답 괜찮은 거 로 골라 보세요
<< 말을 걸어 보세요!
돈이 없어서 싫어요
>> 챗봇 응답 설마 기억 못 하시는 거 예요
<< 말을 걸어 보세요!
q
